In [1750]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [1751]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
#import matplotlib.pyplot as plt
import numpy as np

In [1752]:
class AlexNet(torch.nn.Module):
  def __init__(self):
    super(AlexNet, self).__init__()
    self.conv1 = torch.nn.Conv1d(4, 6, kernel_size=4) 
    self.pool = torch.nn.MaxPool1d(kernel_size=2, stride=2)  
    self.conv2 = torch.nn.Conv1d(6, 16, kernel_size=3)  
    self.fc1 = torch.nn.Linear(376 * 2, 4096)  
    self.fc2 = torch.nn.Linear(4096, 4096) 

  def forward(self, x):
    x = self.pool(torch.nn.functional.relu(self.conv1(x)))  
    x = self.pool(torch.nn.functional.relu(self.conv2(x)))
    x = x.transpose(1, 2)
    x = torch.flatten(x, 1) 
    x = F.relu(self.fc1(x))
    x = F.dropout(x, 0.5)
    x = F.relu(self.fc2(x))
    x = F.dropout(x, 0.5)
    return x

In [1753]:
# Generating random data

train_data = np.load('trainout.npy')
test_data = np.load('testout.npy')


In [1754]:
# Converting the data to tensor type and floating point type

tensor_train_data = tensor_train_data.view(tensor_train_data.size(0),4, -1)
tensor_test_data = tensor_test_data.view(tensor_test_data.size(0),4, -1)


In [1755]:
print(tensor_train_data.shape)
print(tensor_test_data.dtype)
print(tensor_test_data.shape)
print(len(tensor_data))


torch.Size([348915, 4, 198])
torch.float32
torch.Size([93044, 4, 198])
93044


In [1756]:

# Creating random binary labels. and converting it to tensor

label_test = np.random.choice([0, 1], size=len(tensor_test_data))
label_train = np.random.choice([0, 1], size=len(tensor_train_data))

print(label_train.dtype)
label_test = torch.from_numpy(label_test)
label_train = torch.from_numpy(label_train)
print(label_test.dtype)

labels = np.random.choice([0, 1], size=len(tensor_data))
tensor_labels = torch.from_numpy(labels)

train_labels = np.random.choice([0, 1], size=len(tensor_train_data))
tensor_train_labels = torch.from_numpy(train_labels)


int64
torch.int64


In [1757]:
# The most important class, a custom data loader, understand how it is working.

class data_class(Dataset):
    def __init__(self,data,labels):
        self.data=data
        self.labels = labels

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,id):
        data1 = self.data[id]
        label1 = self.labels[id]

        return data1, label1

class train_class(Dataset):
    def __init__(self,data,labels):
        self.data=data
        self.labels = labels

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,id):
        data2 = self.data[id]
        label2 = self.labels[id]

        return data2, label2

In [1758]:
# calling the data_class for the raw random data

train_data=data_class(tensor_train_data,label_train)
test_data=data_class(tensor_test_data,label_test)
dataset = data_class(tensor_data, tensor_labels)

In [1759]:
# Creating the data loader which is going to load the data to the AI model

train_dataloader=DataLoader(train_data,batch_size=2,shuffle=True)
test_dataloader=DataLoader(test_data,batch_size=2,shuffle=True)


In [1760]:
# Instantiating the model and assigning an optimizer to the model and creating a loss function

model=AlexNet().to(device)
optimizer=optim.Adam(params=model.parameters(),lr=0.0001)
loss_fn = nn.CrossEntropyLoss()



In [1761]:
def train(model,device,train_dataloader,optimizer,epochs):
    print("inside train")
    model.train()
    for batch_ids, (img, classes) in enumerate(train_dataloader):
        classes=classes.type(torch.LongTensor)
        img,classes=img.to(device),classes.to(device)
        torch.autograd.set_detect_anomaly(True)     
        optimizer.zero_grad()
        output=model(img)
        loss = loss_fn(output,classes)                
        
        loss.backward()
        optimizer.step()
    if(batch_ids +1) % 2 == 0:
        print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
            epochs, batch_ids* len(img), len(train_dataloader.dataset),
            100.*batch_ids / len(train_dataloader),loss.item()))

In [1762]:
def test(model, device, test_dataloader):
    model.eval()
    test_loss=0
    correct=0
    with torch.no_grad():
        for img,classes in test_dataloader:
            img,classes=img.to(device), classes.to(device)
            y_hat=model(img)
            test_loss+=F.nll_loss(y_hat,classes,reduction='sum').item()
            _,y_pred=torch.max(y_hat,1)
            correct+=(y_pred==classes).sum().item()
        test_loss/=len(test_dataloader)
        print("\n Test set: Avarage loss: {:.0f},Accuracy:{}/{} ({:.0f}%)\n".format(
            test_loss,correct,len(test_dataloader),100.*correct/len(test_dataloader)))
        print('='*30)

In [ ]:
# WE ARE USING RANDOM DATA SO THE TRAINING AND TESTING DOES NOT MATTER, THE AIM IS TO SHOWCASE THE USE OF A CUSTOM DATASET
# SINCE IN PRACTICAL SENSE YOU HAVE TO CLEAN THE DATA AND LOAD THE DATA INTO THE MODEL.


if __name__=='__main__':
    seed=42
    EPOCHS=2
    
    for epoch in range(1,EPOCHS+1):
        train(model,device,train_dataloader,optimizer,epoch)
        test(model,device,test_dataloader)

inside train
